In [182]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd

In [184]:

categories_refined = {
    "前端开发": ["前端", "Web", "前端开发工程师", "JavaScript","HTML", "CSS", "JavaScript", "Vue", "React", "Angular", "H5"],
    "后端开发": ["后端", "Java", "Python开发", "后端工程师", "Node.js", "C#","PHP","net", "软件工程师", "安卓","IOS", "软件开发", "Golang","golang","java", "Android", "iOS", "C++", "JAVA", "NET","ios"],
    "运维": ["运维", "DevOps", "系统管理员", "网络", "信息安全", "管理员", "Linux", "Docker", "Kubernetes", "CI/CD", "自动化部署", "网络安全", "维护", "Dev", "ETL", "OpenStack", "sre", "驻场"],
    "人工智能&分析决策": ["人工智能", "机器学习", "深度学习", "数据分析", "NLP", "算法","视觉", "TensorFlow", "PyTorch", "大模型", "AI", "智能", "分析", "Analyst", "BI", "机器人", "量化", "自然语言处理"]
}

# Function to classify job positions based on job title only
def classify_position_by_title(row):
    title = row['职位']
    for category, keywords in categories_refined.items():
        if any(keyword in title for keyword in keywords):
            return category
    return "其他"


In [198]:
file_path = '/Users/leonliao/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Junzhe/CSE 6242/Project/Updated-Classified-Model-Data-xlsx.csv'
data = pd.read_csv(file_path)

In [203]:
# Retain only the specified columns
columns_to_keep = ['city', 'average_salary', 'education', 'experience', 'company_size', '分类']
filtered_data = data[columns_to_keep]
# Dictionary to translate city names
city_translations = {
    '北京': 'Beijing', '上海': 'Shanghai', '广州': 'Guangzhou', '深圳': 'Shenzhen',
    '杭州': 'Hangzhou', '成都': 'Chengdu', '武汉': 'Wuhan', '重庆': 'Chongqing',
    '南京': 'Nanjing', '西安': 'Xi\'an', '天津': 'Tianjin', '苏州': 'Suzhou'
}

# Dictionary to translate categories
category_translations = {
    '后端开发': 'Backend Development',
    '人工智能&分析决策': 'AI & Analytics',
    '前端开发': 'Frontend Development',
    '移动开发': 'Mobile Development',
    '数据科学': 'Data Science',
    '测试': 'Testing',
    '运维': 'System Admin',
    '网络安全': 'Cybersecurity',
    '游戏开发': 'Game Development',
    "其他": 'Others'
}

# Apply translations and clean up company_size
filtered_data['city'] = filtered_data['city'].map(city_translations).fillna(filtered_data['city'])
filtered_data['分类'] = filtered_data['分类'].map(category_translations).fillna(filtered_data['分类'])
filtered_data['company_size'] = filtered_data['company_size'].str.extract('(\d+-\d+人|\d+人以上)')

# Display the first few rows of the modified dataframe
filtered_data.head()


/var/folders/ct/_hxnz_6n7wqft3v31kp1tyhw0000gn/T/ipykernel_2890/1392714902.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['city'] = filtered_data['city'].map(city_translations).fillna(filtered_data['city'])
/var/folders/ct/_hxnz_6n7wqft3v31kp1tyhw0000gn/T/ipykernel_2890/1392714902.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['分类'] = filtered_data['分类'].map(category_translations).fillna(filtered_data['分类'])
/var/folders/ct/_hxnz_6n7wqft3v31kp1tyhw0000gn/T/ipykernel_2

,city,average_salary,education,experience,company_size,分类
0,Shanghai,12.0,Bachelor,3-5,20-99人,Backend Development
1,Shanghai,42.0,Master,1-3,20-99人,AI & Analytics
2,Shanghai,14.0,Associate,3-5,100-499人,Frontend Development
3,Shanghai,18.0,Associate,unlimited,20-99人,Backend Development
4,Shanghai,15.0,Bachelor,unlimited,10000人以上,Backend Development


In [204]:
# Further clean the company_size column to remove Chinese characters
filtered_data['company_size'] = filtered_data['company_size'].str.extract('(\d+-\d+|\d+人以上)')

# Display the first few rows of the modified dataframe
filtered_data.head()


/var/folders/ct/_hxnz_6n7wqft3v31kp1tyhw0000gn/T/ipykernel_2890/894960022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['company_size'] = filtered_data['company_size'].str.extract('(\d+-\d+|\d+人以上)')


,city,average_salary,education,experience,company_size,分类
0,Shanghai,12.0,Bachelor,3-5,20-99,Backend Development
1,Shanghai,42.0,Master,1-3,20-99,AI & Analytics
2,Shanghai,14.0,Associate,3-5,100-499,Frontend Development
3,Shanghai,18.0,Associate,unlimited,20-99,Backend Development
4,Shanghai,15.0,Bachelor,unlimited,10000人以上,Backend Development


In [205]:
# Replace "10000人以上" with "over 10000"
filtered_data['company_size'] = filtered_data['company_size'].str.replace('人以上', '+').str.replace('10000', 'over 10000')
filtered_data.rename(columns={'分类': 'category'}, inplace=True)
# Display the first few rows of the modified dataframe
filtered_data.head()


/var/folders/ct/_hxnz_6n7wqft3v31kp1tyhw0000gn/T/ipykernel_2890/833337408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['company_size'] = filtered_data['company_size'].str.replace('人以上', '+').str.replace('10000', 'over 10000')
/var/folders/ct/_hxnz_6n7wqft3v31kp1tyhw0000gn/T/ipykernel_2890/833337408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.rename(columns={'分类': 'category'}, inplace=True)


,city,average_salary,education,experience,company_size,category
0,Shanghai,12.0,Bachelor,3-5,20-99,Backend Development
1,Shanghai,42.0,Master,1-3,20-99,AI & Analytics
2,Shanghai,14.0,Associate,3-5,100-499,Frontend Development
3,Shanghai,18.0,Associate,unlimited,20-99,Backend Development
4,Shanghai,15.0,Bachelor,unlimited,over 10000+,Backend Development


In [206]:

# filtered_data.to_csv('model_data.csv', index=False)



In [207]:
data = pd.read_csv('model_data.csv')
data.dropna(inplace=True)

In [208]:
data

,city,average_salary,education,experience,company_size,category
0,Shanghai,12.0,Bachelor,3-5,20-99,Backend Development
1,Shanghai,42.0,Master,1-3,20-99,AI & Analytics
2,Shanghai,14.0,Associate,3-5,100-499,Frontend Development
3,Shanghai,18.0,Associate,unlimited,20-99,Backend Development
4,Shanghai,15.0,Bachelor,unlimited,over 10000+,Backend Development
...,...,...,...,...,...,...
1039,Shenzhen,42.0,Bachelor,5-10,20-99,Others
1040,Shenzhen,21.0,Associate,1-3,20-99,Backend Development
1041,Shenzhen,22.0,Bachelor,3-5,20-99,Backend Development
1042,Shenzhen,14.0,Associate,3-5,500-999,Backend Development


In [209]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Encode categorical variables
le_education = LabelEncoder()
le_experience = LabelEncoder()
le_company_type = LabelEncoder()
le_company_size = LabelEncoder()
le_category = LabelEncoder()
le_city = LabelEncoder()  # 创建一个新的LabelEncoder对象用于城市变量的编码

data['education'] = le_education.fit_transform(data['education'])
data['experience'] = le_experience.fit_transform(data['experience'])
data['company_size'] = le_company_size.fit_transform(data['company_size'])
data['category'] = le_category.fit_transform(data['category'])
data['city'] = le_city.fit_transform(data['city'])




In [171]:
data.head()

,city,average_salary,education,experience,company_size,category
0,2,12.0,1,1,3,1
1,2,42.0,3,0,3,0
2,2,14.0,0,1,1,2
3,2,18.0,0,4,3,1
4,2,15.0,1,4,5,1


In [177]:
# Prepare features and target
X = data.drop('average_salary', axis=1)
y = data['average_salary']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.6, learning_rate = 0.05,
                             max_depth = 9, alpha = 29, n_estimators = 1000)

xgb_model.fit(X_train, y_train)

# Calculate training and testing accuracy
training_accuracy = xgb_model.score(X_train, y_train)
testing_accuracy = xgb_model.score(X_test, y_test)

training_accuracy, testing_accuracy

(0.5680700935002287, 0.44698200663766563)

In [173]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
lr_score = r2_score(y_test, y_pred)
lr_score

0.054911217262161305

In [174]:
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr_model.fit(X_train, y_train)
y_pred_svr = svr_model.predict(X_test)
svr_score = r2_score(y_test, y_pred_svr)

print(svr_score)

0.3027228741559391


In [175]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np

# 转换数据为 PyTorch 张量
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# 定义包含两个额外隐藏层的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        return out

# 初始化模型
input_size = X_train.shape[1]
hidden_size1 = 128
hidden_size2 = 64
hidden_size3 = 32
output_size = 1
model = SimpleNN(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 训练模型
num_epochs = 10000
for epoch in range(num_epochs):
    inputs = Variable(X_train_tensor)
    targets = Variable(y_train_tensor)

    optimizer.zero_grad()
    outputs = model(inputs)

    loss = criterion(outputs, targets)
    loss.backward()

    optimizer.step()

    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# 在测试集上进行预测
with torch.no_grad():
    inputs = Variable(X_test_tensor)
    outputs = model(inputs)
    predicted = outputs.numpy()

# 计算神经网络模型的 R^2 分数
nn_score = r2_score(y_test, predicted)

print("神经网络模型的 R^2 分数:", nn_score)


Epoch [100/10000], Loss: 88.9397
Epoch [200/10000], Loss: 72.1931
Epoch [300/10000], Loss: 64.4460
Epoch [400/10000], Loss: 56.6289
Epoch [500/10000], Loss: 53.9485
Epoch [600/10000], Loss: 49.9830
Epoch [700/10000], Loss: 46.9862
Epoch [800/10000], Loss: 45.9296
Epoch [900/10000], Loss: 46.5407
Epoch [1000/10000], Loss: 42.1466
Epoch [1100/10000], Loss: 40.4765
Epoch [1200/10000], Loss: 38.7936
Epoch [1300/10000], Loss: 38.5895
Epoch [1400/10000], Loss: 37.8519
Epoch [1500/10000], Loss: 37.1650
Epoch [1600/10000], Loss: 36.8136
Epoch [1700/10000], Loss: 36.5725
Epoch [1800/10000], Loss: 36.4618
Epoch [1900/10000], Loss: 39.0899
Epoch [2000/10000], Loss: 37.1144
Epoch [2100/10000], Loss: 36.6079
Epoch [2200/10000], Loss: 36.3087
Epoch [2300/10000], Loss: 36.1271
Epoch [2400/10000], Loss: 36.0057
Epoch [2500/10000], Loss: 36.1887
Epoch [2600/10000], Loss: 35.8736
Epoch [2700/10000], Loss: 35.8227
Epoch [2800/10000], Loss: 109.2738
Epoch [2900/10000], Loss: 36.1994
Epoch [3000/10000], Lo

In [178]:
from joblib import dump

# 假设 `pipeline` 是您的训练好的模型
dump(xgb_model, 'model.joblib')

['model.joblib']

In [179]:
xgb_model.predict(X_test)

array([13.29599  , 14.496778 , 28.828856 ,  9.042927 , 16.393763 ,
       15.266108 , 22.913143 , 25.888048 , 32.337803 , 15.790975 ,
       26.305872 , 16.885147 , 11.982718 , 26.38901  , 28.384853 ,
       20.499508 , 21.86034  , 24.076729 , 22.072691 , 25.601196 ,
       14.5968485, 13.390535 ,  8.1468725, 34.92166  , 32.08222  ,
       21.8331   , 24.336555 , 18.1537   ,  6.2224646, 31.399145 ,
        9.77819  , 22.193071 , 13.306944 , 17.193356 , 10.426996 ,
       29.341473 , 16.968834 , 17.853333 , 30.32078  , 16.016003 ,
       25.1035   , 37.28286  , 30.569426 , 20.833462 , 34.246197 ,
       22.478685 , 15.742665 , 32.25887  , 21.108425 , 15.790975 ,
       26.972847 , 25.597563 , 10.488169 , 10.168922 , 18.29322  ,
       12.154493 , 10.426996 , 14.1301155, 22.212236 , 18.387205 ,
       29.621489 , 34.70946  , 13.204463 , 10.488169 , 42.146812 ,
       10.785811 , 18.287891 , 16.654955 , 32.337803 , 12.9721775,
       32.08222  , 11.360853 , 22.212236 , 14.845339 , 17.4877

In [210]:
education_classes = le_education.classes_
experience_classes = le_experience.classes_
company_size_classes = le_company_size.classes_
category_classes = le_category.classes_
city_classes = le_city.classes_

print("Education Classes:", education_classes)
print("Experience Classes:", experience_classes)
print("Company Size Classes:", company_size_classes)
print("Category Classes:", category_classes)
print("City Classes:", city_classes)

Education Classes: ['Associate' 'Bachelor' 'Doctor' 'Master' 'Unlimited']
Experience Classes: ['1-3' '3-5' '5-10' 'over 10' 'unlimited']
Company Size Classes: ['0-20' '100-499' '1000-9999' '20-99' '500-999' 'over 10000+']
Category Classes: ['AI & Analytics' 'Backend Development' 'Frontend Development' 'Others'
 'System Admin']
City Classes: ['Beijing' 'Guangzhou' 'Shanghai' 'Shenzhen']


In [212]:
education_mapping = dict(zip(le_education.transform(le_education.classes_), le_education.classes_))
experience_mapping = dict(zip(le_experience.transform(le_experience.classes_), le_experience.classes_))
company_size_mapping = dict(zip(le_company_size.transform(le_company_size.classes_), le_company_size.classes_))
category_mapping = dict(zip(le_category.transform(le_category.classes_), le_category.classes_))
city_mapping = dict(zip(le_city.transform(le_city.classes_), le_city.classes_))

print("Education Mapping:", education_mapping)
print("Experience Mapping:", experience_mapping)
print("Company Size Mapping:", company_size_mapping)
print("Category Mapping:", category_mapping)
print("City Mapping:", city_mapping)

Education Mapping: {0: 'Associate', 1: 'Bachelor', 2: 'Doctor', 3: 'Master', 4: 'Unlimited'}
Experience Mapping: {0: '1-3', 1: '3-5', 2: '5-10', 3: 'over 10', 4: 'unlimited'}
Company Size Mapping: {0: '0-20', 1: '100-499', 2: '1000-9999', 3: '20-99', 4: '500-999', 5: 'over 10000+'}
Category Mapping: {0: 'AI & Analytics', 1: 'Backend Development', 2: 'Frontend Development', 3: 'Others', 4: 'System Admin'}
City Mapping: {0: 'Beijing', 1: 'Guangzhou', 2: 'Shanghai', 3: 'Shenzhen'}
